# Lab 2.1
Пользуясь выражением для погрешности интерполирования, определить шаг h в составной квадратурной формуле средних прямоугольников, которая обеспечит требуемую точность результата.

In [9]:
import math

# Заданные параметры
a = 0.7
b = 1.7
epsilon = 1e-5

# Производная функции: f'(x) = 0.7e^x + 0.3cos(x)
# Найдём максимум производной на отрезке [a, b]
max_f_derivative = 0.7 * math.exp(b) + 0.3 * 1  # cos(x) <= 1

# Вычислим необходимый шаг h
h = (2 * epsilon) / ((b - a) * max_f_derivative)

# Количество интервалов (округляем вверх)
n = math.ceil((b - a) / h)

# Перерасчитаем шаг с учётом n
h = (b - a) / n

# Метод правых прямоугольников
def f(x):
    return 0.7 * math.exp(x) + 0.3 * math.sin(x)

def true_integral(a, b):
    return 0.7 * math.exp(b) - 0.3 * math.cos(b) - (0.7 * math.exp(a) - 0.3 * math.cos(a))

integral = 0
for i in range(1, n + 1):
    x_i = a + i * h  # правая граница i-го отрезка
    integral += f(x_i)

integral *= h
true_val = true_integral(a,b)
error = abs(true_val - integral)
error_estimate = ((b - a)**2 / 2) * max_f_derivative * h

print(f"Интеграл ≈ {integral:.8f}")
print(f'Истинное значение = {true_val}')
print(f'Истинная погрешность - {error}')
print(f'Оценка погрешности - {error_estimate}')
print(f'Использовано разбиений N - {n}')
print(f'Величина шага h - {h}')

Интеграл ≈ 2.69024840
Истинное значение = 2.69024228345371
Истинная погрешность - 6.114488520836403e-06
Оценка погрешности - 9.999959277137309e-06
Использовано разбиений N - 206589
Величина шага h - 4.840528779363857e-06


# Lab 2.2.
Для СКФ из п.1 определить величину h шага разбиения исходного отрезка интегрирования, достаточного для достижения точности  $\varepsilon$, по правилу Рунге.

In [10]:
import math

# Заданная функция
def f(x):
    return 0.7 * math.exp(x) + 0.3 * math.sin(x)

# Метод правых прямоугольников
def right_rectangle_integral(a, b, n):
    h = (b - a) / n
    result = 0.0
    for i in range(1, n + 1):
        x = a + i * h
        result += f(x)
    return result * h

# Метод Рунге для правых прямоугольников
def runge_right_rectangle(a, b, epsilon):
    p = 1  # порядок точности для правых прямоугольников
    N1 = 5
    N2 = 2 * N1

    q1 = right_rectangle_integral(a, b, N1)
    q2 = right_rectangle_integral(a, b, N2)

    while True:
        R = abs(q2 - q1) / (1 - (N1 / N2))  # эквивалентно R = 2 * |q2 - q1|, так как N2 = 2 * N1

        if R <= epsilon:
            return q2, (b - a) / N2, N2, R  # значение интеграла, шаг, число разбиений, погрешность

        N1 = N2
        N2 = 2 * N2
        q1 = q2
        q2 = right_rectangle_integral(a, b, N2)

# Параметры
a = 0.7
b = 1.7
epsilon = 1e-5

# Запуск алгоритма
interpolated, h_final, N_final, error = runge_right_rectangle(a, b, epsilon)
true_val = true_integral(a,b)
true_error_2 = abs(true_val - interpolated)

# Вывод
print(f"Результат интегрирования:     {interpolated:.8f}")
print(f'Истинное значение             {true_val:.8f}')
print(f'Истинная ошибка:              {true_error_2}')
print(f"Итоговый шаг h:               {h_final}")
print(f"Число отрезков N:             {N_final}")
print(f"Оценка погрешности по Рунге:  {error:.2e}")


Результат интегрирования:     2.69024614
Истинное значение             2.69024228
Истинная ошибка:              3.854936988734181e-06
Итоговый шаг h:               3.0517578125e-06
Число отрезков N:             327680
Оценка погрешности по Рунге:  7.71e-06


# Lab 2.3.
Применить квадратурную формулу НАСТ Гаусса при указанном значении n. Оценить погрешность интегрирования через формулу остаточного члена $R_n(f)$, n = 3.

In [24]:
import math
import numpy as np
from scipy.integrate import quad
from numpy.polynomial.legendre import leggauss
from scipy.optimize import minimize_scalar

# 10-я производная (совпадает с f)
def f_10_derivative(x):
    return 0.7 * math.exp(x) - 0.3 * math.sin(x)

# Гаусс-КФ на [a, b] с произвольным n
def gauss_integrate(f, a, b, n):
    # Вычисление узлов и весов Гаусса на [-1, 1]
    nodes, weights = leggauss(n)

    # Преобразуем на [a, b]
    half = (b - a) / 2
    mid = (a + b) / 2

    result = 0
    x_mapped = []
    for xi, wi in zip(nodes, weights):
        x_mapped.append(half * xi + mid)
        result += wi * f(x_mapped[-1])

    return half * result, nodes, weights, x_mapped

# Оценка остаточного члена R_n(f)
def estimate_gauss_error(a, b, n):
    # Максимум f^(10)(x) ≈ максимум f(x) на [a, b]
    #x_vals = [a, (a + b) / 2, b]
    res = minimize_scalar(lambda x: -abs(f_10_derivative(x)), bounds=(a, b), method='bounded')
    max_f10 = f_10_derivative(res.x)

    # Часть на [-1, 1]
    numerator = 2**(2*n + 3)
    denominator = (2*n + 3) * math.factorial(2*n + 2)
    factor = (math.factorial(n + 1)**2 / math.factorial(2*n + 2))**2

    Rn_standard = (numerator / denominator) * factor * max_f10

    # Переход на [a, b]
    scale = ((b - a) / 2)**(2*n + 3)
    Rn_scaled = scale * Rn_standard
    return Rn_scaled

# Основной расчёт
a, b = 0.7, 1.7
n = 4

approx, x, A, x_mapped = gauss_integrate(f, a, b, n)
true_value, _ = quad(f, a, b)
true_error = abs(approx - true_value)
error_estimate = estimate_gauss_error(a, b, n)

# Вывод
print(f"Приближённое значение (Гаусс, n={n}): {approx:.10f}")
print(f"Точное значение (quad):               {true_value:.10f}")
print(f"Истинная погрешность:                 {true_error:.2e}")
print(f"Оценка остаточного члена:             {error_estimate:.2e}")
print(f'Корни Xk - {x_mapped}')
print(f'Коэффициенты Аk - {A}')
print(f'Приведенные Xk - {x}')


Приближённое значение (Гаусс, n=4): 2.6902422820
Точное значение (quad):               2.6902422835
Истинная погрешность:                 1.47e-09
Оценка остаточного члена:             1.39e-12
Корни Xk - [0.7694318442029737, 1.030009478207572, 1.369990521792428, 1.6305681557970262]
Коэффициенты Аk - [0.34785485 0.65214515 0.65214515 0.34785485]
Приведенные Xk - [-0.86113631 -0.33998104  0.33998104  0.86113631]


# Lab 2.4.
Провести сравнительный анализ полученных результатов.